In [1]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


# 构建简单的模型
## 序列模型
在 Keras 中，您可以通过组合层来构建模型。模型（通常）是由层构成的图。最常见的模型类型是层的堆叠：`tf.keras.Sequential` 模型。

要构建一个简单的全连接网络（即多层感知器），请运行以下代码：

In [2]:
model = tf.keras.Sequential()
# Adds a densely-connected alyer with 64 units to the model:
model.add(layers.Dense(64,activation='relu'))
# Add another:
model.add(layers.Dense(64,activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10,activation='softmax'))

# 配置层
我们可以使用很多 tf.keras.layers，它们具有一些相同的构造函数参数：

* `activation`：设置层的激活函数。此参数由内置函数的名称指定，或指定为可调用对象。默认情况下，系统不会应用任何激活函数。
* `kernel_initializer` 和 `bias_initializer`：创建层权重（核和偏差）的初始化方案。此参数是一个名称或可调用对象，默认为 "Glorot uniform" 初始化器。
* `kernel_regularizer` 和 `bias_regularizer`：应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。

以下代码使用构造函数参数实例化 `tf.keras.layers. Dense` 层：

In [3]:
# Create a sigmoid layer:
layers.Dense(64,activation='sigmoid')
# Or:
layers.Dense(64,activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernelmatrix:
layers.Dense(64,kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regulariztion of factor 0.01 applied to the bias vector:
layers.Dense(64,bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64,kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64,bias_initializer=tf.keras.initializers.constant(2.0))

# 训练和评估
## 设置训练流程
构建好模型后，通过调用 `compile` 方法配置该模型的学习流程：

In [5]:
model = tf.keras.Sequential([
    # Adds  a densely-connected alyer with 64 units to the model:
    layers.Dense(64,activation='relu'),
    # Add another:
    layers.Dense(64,activation='relu'),
    # Add a softmax layer with 10 output units:
    layers.Dense(10,activation='softmax')
])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

`tf.keras.Model.compile` 采用三个重要参数：

* `optimizer`：此对象会指定训练过程。从 `tf.train` 模块向其传递优化器实例，例如 `tf.train.AdamOptimizer`、`tf.train.RMSPropOptimizer` 或 `tf.train.GradientDescentOptimizer`。
* `loss`：要在优化期间最小化的函数。常见选择包括均方误差 (mse)、`categorical_crossentropy` 和 `binary_crossentropy`。损失函数由名称或通过从 `tf.keras.losses` 模块传递可调用对象来指定。
* `metrics`：用于监控训练。它们是 `tf.keras.metrics` 模块中的字符串名称或可调用对象。

以下代码展示了配置模型以进行训练的几个示例：

In [6]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
             loss='mse', # mean squared error
             metrics=['mae']) # mean absolute error

# Configure a model for categorical classification
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

# 输入 NumPy 数据
对于小型数据集，请使用内存中的 **NumPy**数组训练和评估模型。使用 `fit` 方法使模型与训练数据“拟合”：

In [7]:
import numpy as np

data = np.random.random((1000,32))
labels = np.random.random((1000,10))

model.fit(data,labels,epochs=10,batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 0s 179us/sample - loss: 11.5973 - categorical_accuracy: 0.0910
Epoch 2/10
1000/1000 [==============================] - 0s 31us/sample - loss: 11.5545 - categorical_accuracy: 0.0970
Epoch 3/10
1000/1000 [==============================] - 0s 29us/sample - loss: 11.5404 - categorical_accuracy: 0.0970
Epoch 4/10
1000/1000 [==============================] - 0s 29us/sample - loss: 11.5349 - categorical_accuracy: 0.1140
Epoch 5/10
1000/1000 [==============================] - 0s 27us/sample - loss: 11.5343 - categorical_accuracy: 0.1200
Epoch 6/10
1000/1000 [==============================] - 0s 27us/sample - loss: 11.5306 - categorical_accuracy: 0.1050
Epoch 7/10
1000/1000 [==============================] - 0s 27us/sample - loss: 11.5268 - categorical_accuracy: 0.1480
Epoch 8/10
1000/1000 [==============================] - 0s 27us/sample - loss: 11.5293 - categorical_accuracy: 0.1020
Epoch 9

`tf.keras.Model.fit` 采用三个重要参数：

- `epochs`：以周期为单位进行训练。一个周期是对整个输入数据的一次迭代（以较小的批次完成迭代）。
- `batch_size`：当传递 NumPy 数据时，模型将数据分成较小的批次，并在训练期间迭代这些批次。此整数指定每个批次的大小。请注意，如果样本总数不能被批次大小整除，则最后一个批次可能更小。
- `validation_data`：在对模型进行原型设计时，您需要轻松监控该模型在某些验证数据上达到的效果。传递此参数（输入和标签元组）可以让该模型在每个周期结束时以推理模式显示所传递数据的损失和指标。
下面是使用 `validation_data` 的示例：

In [8]:
import numpy as np 

data = np.random.random((1000,32))
labels = np.random.random((1000,10))

val_data = np.random.random((100,32))
val_labels = np.random.random((100,10))

model.fit(data,labels,epochs=10,batch_size=32,
         validation_data=(val_data,val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 76us/sample - loss: 11.5073 - categorical_accuracy: 0.1110 - val_loss: 11.6678 - val_categorical_accuracy: 0.1200
Epoch 2/10
1000/1000 [==============================] - 0s 31us/sample - loss: 11.5025 - categorical_accuracy: 0.1110 - val_loss: 11.6724 - val_categorical_accuracy: 0.0900
Epoch 3/10
1000/1000 [==============================] - 0s 32us/sample - loss: 11.5018 - categorical_accuracy: 0.1230 - val_loss: 11.6722 - val_categorical_accuracy: 0.0600
Epoch 4/10
1000/1000 [==============================] - 0s 31us/sample - loss: 11.4989 - categorical_accuracy: 0.1170 - val_loss: 11.6749 - val_categorical_accuracy: 0.1300
Epoch 5/10
1000/1000 [==============================] - 0s 31us/sample - loss: 11.4983 - categorical_accuracy: 0.1460 - val_loss: 11.6924 - val_categorical_accuracy: 0.0600
Epoch 6/10
1000/1000 [==============================] - 0s 32us/sample - loss: 11.4959 -

### 输入 tf.data 数据集
使用 Datasets API 可扩展为大型数据集或多设备训练。将 `tf.data.Dataset` 实例传递到 `fit` 方法：

In [9]:
# Instantiates a toy dataset instance:
dataset = tf.data.Dataset.from_tensor_slices((data,labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

#Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
model.fit(dataset,epochs=10,steps_per_epoch=30)

Epoch 1/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4902 - categorical_accuracy: 0.1583
Epoch 2/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4785 - categorical_accuracy: 0.1635
Epoch 3/10
30/30 [==============================] - 0s 1000us/step - loss: 11.4506 - categorical_accuracy: 0.1560
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4685 - categorical_accuracy: 0.1549
Epoch 5/10
30/30 [==============================] - 0s 967us/step - loss: 11.4747 - categorical_accuracy: 0.1624
Epoch 6/10
30/30 [==============================] - 0s 967us/step - loss: 11.4391 - categorical_accuracy: 0.1709
Epoch 7/10
30/30 [==============================] - 0s 933us/step - loss: 11.4290 - categorical_accuracy: 0.1667
Epoch 8/10
30/30 [==============================] - 0s 933us/step - loss: 11.4215 - categorical_accuracy: 0.1709
Epoch 9/10
30/30 [==============================] - 0s 967us/step - loss: 11.4201 - categorical_accur

在上方代码中，fit 方法使用了 `steps_per_epoch` 参数（表示模型在进入下一个周期之前运行的训练步数）。由于 `Dataset` 会生成批次数据，因此该代码段不需要 `batch_size`。

数据集也可用于验证：

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,
          validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 6ms/step - loss: 11.4409 - categorical_accuracy: 0.1885 - val_loss: 11.7262 - val_categorical_accuracy: 0.1146
Epoch 2/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4232 - categorical_accuracy: 0.1923 - val_loss: 11.9538 - val_categorical_accuracy: 0.0735
Epoch 3/10
30/30 [==============================] - 0s 1ms/step - loss: 11.3964 - categorical_accuracy: 0.2019 - val_loss: 12.0128 - val_categorical_accuracy: 0.0441
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4138 - categorical_accuracy: 0.2094 - val_loss: 12.1574 - val_categorical_accuracy: 0.0735
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4239 - categorical_accuracy: 0.2030 - val_loss: 11.7291 - val_categorical_accuracy: 0.1146
Epoch 6/10
30/30 [==============================] - 0s 1000us/step - loss: 11.3931 - categorical_accuracy: 0.2094 - val_loss: 11.9984 - val_categorical_accuracy: 0.0735
E

### 评估和预测
`tf.keras.Model.evaluate` 和 `tf.keras.Model.predict` 方法可以使用 **NumPy** 数据和 `tf.data.Dataset`。

要评估所提供数据的推理模式损失和指标，请运行以下代码：

In [12]:
data = np.random.random((1000,32))
labels = np.random.random((1000,10))

model.evaluate(data,labels,batch_size=32)

model.evaluate(dataset, steps=30)

30/30 [==============================] - 0s 2ms/step - loss: 11.4453 - categorical_accuracy: 0.1906


[11.445285511016845, 0.190625]

要在所提供数据（采用 NumPy 数组形式）的推理中预测最后一层的输出，请运行以下代码：

In [14]:
result = model.predict(data,batch_size=32)
print(result.shape)

(1000, 10)
